In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [2]:
import datasets
import pandas as pd
import numpy as np
import torch
import random
from tqdm import tqdm

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Load the CNN/DailyMail dataset
cnn_dataset = datasets.load_dataset("abisee/cnn_dailymail", "3.0.0")

# Convert to pandas DataFrame for easier manipulation
train_df = pd.DataFrame(cnn_dataset["train"])
val_df = pd.DataFrame(cnn_dataset["validation"])
test_df = pd.DataFrame(cnn_dataset["test"])

# Sample 1% of the training data
sample_size = int(len(train_df) * 0.001)
train_sample = train_df.sample(n=sample_size, random_state=42)

print(f"Full training set size: {len(train_df)}")
print(f"1% sample size: {len(train_sample)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Full training set size: 287113
1% sample size: 287


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def tokenize_document(document):
    # Split document into sentences
    sentences = sent_tokenize(document)

    # Preprocess each sentence
    processed_sentences = [preprocess_text(sentence) for sentence in sentences]

    # Remove empty sentences
    processed_sentences = [s for s in processed_sentences if s.strip()]

    return processed_sentences

# Apply preprocessing to the sampled data
train_sample['processed_article'] = train_sample['article'].apply(tokenize_document)
train_sample['processed_highlights'] = train_sample['highlights'].apply(tokenize_document)

# Create labels for extractive summarization (1 if sentence is in highlights, 0 otherwise)
def create_extractive_labels(article_sentences, highlight_sentences):
    labels = []
    for sentence in article_sentences:
        # Check if this sentence is similar to any highlight sentence
        is_in_highlights = any(
            similarity_score(sentence, highlight) > 0.7
            for highlight in highlight_sentences
        )
        labels.append(1 if is_in_highlights else 0)
    return labels

def similarity_score(sent1, sent2):
    # Simple word overlap similarity
    words1 = set(word_tokenize(sent1))
    words2 = set(word_tokenize(sent2))

    if not words1 or not words2:
        return 0

    overlap = len(words1.intersection(words2))
    return overlap / max(len(words1), len(words2))

# Create extractive labels
train_sample['extractive_labels'] = [
    create_extractive_labels(article, highlight)
    for article, highlight in zip(train_sample['processed_article'], train_sample['processed_highlights'])
]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9310e153eba0d746f9a67647f309587f4d5baa19496421afa8e0348e91283eb0
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [5]:
from datasets import Dataset,DatasetDict
from sklearn.model_selection import train_test_split


# Split into train and validation sets (90% train, 10% validation)
train_df, val_df = train_test_split(train_sample, test_size=0.1, random_state=42)
# Convert Pandas DataFrames to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_dict({
        "input_text": train_df["processed_article"].tolist(),
        "target_text": train_df["processed_highlights"].tolist(),
    }),
    "validation": Dataset.from_dict({
        "input_text": val_df["processed_article"].tolist(),
        "target_text": val_df["processed_highlights"].tolist(),
    })
})

# Extract train and validation datasets
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [6]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
# Load tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# Define tokenization function
def tokenize_function(batch):
    # Join the list of sentences into a single string for input
    input_text = [" ".join(sentences) for sentences in batch["input_text"]]
    target_text = [" ".join(sentences) for sentences in batch["target_text"]]

    inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(target_text, padding="max_length", truncation=True, max_length=128)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Load pre-trained BART model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Map:   0%|          | 0/258 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [15]:
import os
os.environ["WANDB_DISABLED"] = "false"
import wandb
wandb.init(project ="Transformer only")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    evaluation_strategy="epoch",  # Enables evaluation every epoch
    save_strategy="epoch",        # Saves model every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=0,  # Limits saved checkpoints to avoid storage issues
    load_best_model_at_end=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,  # Include validation set
)

# Train the model
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: b22cs093 (b22cs093-prom-iit-rajasthan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.039690
2,No log,1.042751
3,No log,1.140092


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=195, training_loss=0.4286028739733574, metrics={'train_runtime': 710.6588, 'train_samples_per_second': 1.089, 'train_steps_per_second': 0.274, 'total_flos': 838669481017344.0, 'train_loss': 0.4286028739733574, 'epoch': 3.0})

In [28]:
from rouge_score import rouge_scorer

# Ensure model is in evaluation mode
model.eval()

# ROUGE Evaluation Function
def evaluate_summaries(generated_summaries, reference_summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for gen_sum, ref_sum in zip(generated_summaries, reference_summaries):
        score = scorer.score(ref_sum, gen_sum)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    avg_scores = {key: sum(values) / len(values) if values else 0.0 for key, values in scores.items()}
    return avg_scores

# Generate summaries for a small test set
test_sample = val_df.head(10)  # Using validation set
generated_summaries = []
reference_summaries = []

with torch.no_grad():  # Disable gradient tracking for inference
    for _, row in test_sample.iterrows():
        input_text = row['article']
        input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids.to(model.device)
        output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        generated_summaries.append(generated_summary)
        reference_summaries.append(row['highlights'])

# Calculate ROUGE scores
rouge_scores = evaluate_summaries(generated_summaries, reference_summaries)


# Display ROUGE scores
print("\nROUGE Scores (Fine-tuned BART Summaries):")
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")





ROUGE Scores (Fine-tuned BART Summaries):
ROUGE-1: 0.3820
ROUGE-2: 0.1750
ROUGE-L: 0.2700
